In [1]:
import ray
import os
import time
import numpy as np
import requests

address = ray.init()

2020-01-21 13:22:11,795	INFO resource_spec.py:212 -- Starting Ray with 3.81 GiB memory available for workers and up to 1.91 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-01-21 13:22:12,103	INFO services.py:1093 -- View the Ray dashboard at localhost:8265


### Debug blocked actor creation tasks

In [2]:
@ray.remote(resources={"Custom": 1}, num_cpus=0)
class A(object):
    def __init__(self):
        pass
    
    def f(self):
        return 0
        
@ray.remote
class B(object):
    def __init__(self, x, y):
        self.a = A.remote()
        
    def f(self):
        return ray.get(self.a.f.remote())

b = B.remote(3, y=5)

try:
    ray.get(b.f.remote(), timeout=2)
except ray.exceptions.RayTimeoutError:
    print("Session hangs because actor A cannot be created. ")

2020-01-21 13:22:19,140	WARNING worker.py:1063 -- The actor or task with ID ffffffffffffffff2512146c0100 is infeasible and cannot currently be scheduled. It requires {Custom: 1.000000} for execution and {Custom: 1.000000} for placement, however there are no nodes in the cluster that can provide the requested resources. To resolve this issue, consider reducing the resource requests of this task or add nodes that can fit the task.


Session hangs because actor A cannot be created. 


### Local memory usage

- Local memory info
    - Number of object ids in scope
    - Number of local objects
    - Used Object Memory
    
In the example below, all objects (strings) are stored locally. 

In [3]:
@ray.remote
def f():
    return "hello world!"

@ray.remote
class A(object):
    def __init__(self):
        self.object_ids = []
    
    def f(self):
        for idx in range(50):
            ray.show_in_webui("Loop index = {}...".format(idx))
            self.object_ids.append(f.remote())
            time.sleep(0.5)

a = A.remote()
_ = a.f.remote()

ObjectID(44ee453cd1e8e283f66d17ba010000c801000000)

### Inspect node memory usage

- Node memory info
    - Number of object ids in scope
    - Used object store memory
    
In the example below, all objects (strings) are stored on the node. 

In [4]:
@ray.remote
class C(object):
    def __init__(self):
        self.object_ids = []
    
    def push(self):
        object_id = ray.put("test")
        self.object_ids.append(object_id)
        time.sleep(1)
        return object_id
    
    def clean_memory(self):
        del self.object_ids
        
@ray.remote
class D(object):
    def __init__(self):
        self.object_ids = []
        
    def fetch(self):
        c = C.remote()
        
        for idx in range(20):
            ray.show_in_webui("Loop index = {}...".format(idx))
            time.sleep(0.5)
            object_id = ray.get(c.push.remote())
            self.object_ids.append(object_id)  

    def clean_memory(self):
        del self.object_ids
        
d = D.remote()
_ = d.fetch.remote()

The following command clears out the number of object ids in scope for actor `d`. 

In [5]:
_ = d.clean_memory.remote()

### Profile python program with py-spy

In [7]:
import urllib.parse

def operation_add():
    t = time.time()
    x = 0
    while time.time() - t < 20:
        x += 1
    return x

@ray.remote
class A(object):
    def __init__(self):
        pass
    
    def test(self):
        for _ in range(10):
            operation_add()
        
    def getpid(self):
        return os.getpid()

a = A.remote()
actor_pid = ray.get(a.getpid.remote())
a.test.remote()

webui_url = address["webui_url"]
launch_profiling_url = "http://{}/api/launch_profiling?node_id={}&pid={}&duration=10".format(webui_url, ray.nodes()[0]["NodeID"], actor_pid)
profiling_id = requests.get(launch_profiling_url).json()
while True:
    check_status_url = "http://{}/api/check_profiling_status?profiling_id={}".format(webui_url, profiling_id)
    status = requests.get(check_status_url).json()
    if status["status"] == "finished":
        profiling_url = "http://{}/api/get_profiling_info?profiling_id={}".format(webui_url, profiling_id)
        encoded_profile_url = urllib.parse.quote(profiling_url)
        print("http://{}/speedscope/index.html#profileURL={}".format(webui_url, encoded_profile_url))
        break

http://localhost:8265/speedscope/index.html#profileURL=http%3A//localhost%3A8265/api/get_profiling_info%3Fprofiling_id%3D5f8c5ce3-df1a-47ea-b666-b4ae567aaeae


In [19]:
ray.shutdown(); ray.init()

2020-01-20 16:25:09,308	INFO resource_spec.py:212 -- Starting Ray with 3.37 GiB memory available for workers and up to 1.7 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-01-20 16:25:09,547	INFO services.py:1093 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '192.168.1.69',
 'redis_address': '192.168.1.69:12211',
 'object_store_address': '/tmp/ray/session_2020-01-20_16-25-09_296736_92770/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-01-20_16-25-09_296736_92770/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-01-20_16-25-09_296736_92770'}

### Example 1: Monitor distributed actors

Reference: github issue #3609, @EricSteinberger

In [8]:
import torch


class NeuralNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.l = torch.nn.Linear(1000, 2048)
        self.l2 = torch.nn.Linear(2048, 2)

    def forward(self, x):
        return self.l2(self.l(x))


@ray.remote(num_cpus=1)
class TestActor:
    def __init__(self):
        self.net = NeuralNet()

    def test(self, batch_size):
        p = self.net(torch.rand((batch_size, 1000),))
        
def test(num_actors):
    t = time.time()
    actors = [TestActor.remote() for _ in range(num_actors)]

    t = time.time()
    for _ in range(5000//num_actors):
        ray.get([actor.test.remote(128) for actor in actors])
    
    print(f"Test: num_actors = {num_actors}, time = {time.time() - t}")


# %env OMP_NUM_THREADS=1

test(num_actors=1)
test(num_actors=4)

Test: num_actors = 1, time = 18.386929988861084
Test: num_actors = 4, time = 10.957993030548096


### Example 2: Distributed network training

Reference: github issue #6633, @JaeLiiin

In [9]:
%%capture

from tensorflow.keras import layers
import json


def create_keras_model():
    import tensorflow as tf
    model = tf.keras.Sequential()
    # Adds a densely-connected layer with 64 units to the model:
    model.add(layers.Dense(64, activation="relu", input_shape=(32, )))
    # Add another:
    model.add(layers.Dense(64, activation="relu"))
    # Add a softmax layer with 10 output units:
    model.add(layers.Dense(10, activation="softmax"))

    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(0.01),
        loss=tf.keras.losses.categorical_crossentropy,
        metrics=[tf.keras.metrics.categorical_accuracy])
    return model


def random_one_hot_labels(shape):
    n, n_class = shape
    classes = np.random.randint(0, n_class, n)
    labels = np.zeros((n, n_class))
    labels[np.arange(n), classes] = 1
    return labels


@ray.remote
class Network(object):
    def __init__(self):
        self.model = create_keras_model()
        self.dataset = np.random.random((1000, 32))
        self.labels = random_one_hot_labels((1000, 10))

    def train(self):
        history = self.model.fit(self.dataset, self.labels, verbose=False)
        time.sleep(0.5)
        ray.show_in_webui(repr(history.history))
        return history.history

    def get_weights(self):
        return self.model.get_weights()

    def set_weights(self, weights):
        # Note that for simplicity this does not handle the optimizer state.
        self.model.set_weights(weights)

In [10]:
%%capture

result_object_ids = []
result2_object_ids = []

NetworkActor = Network.remote()
NetworkActor2 = Network.remote()

for itr in range(20):
    weights = ray.get(
        [NetworkActor.get_weights.remote(),
         NetworkActor2.get_weights.remote()])

    averaged_weights = [(layer1 + layer2) / 2
                        for layer1, layer2 in zip(weights[0], weights[1])]

    weight_id = ray.put(averaged_weights)
    [
        actor.set_weights.remote(weight_id)
        for actor in [NetworkActor, NetworkActor2]
    ]
    result_object_ids.append(NetworkActor.train.remote())
    result2_object_ids.append(NetworkActor2.train.remote())

### Example 3: monitor MNIST training with tune
- Actor construction which exposes parameter configuration
- Task execution
    - Number of tasks executed
    - Function descriptor of currently executed task
    - Number of pending tasks listed on the task queue
- Training accuracy shown as actor message

Reference: ray docs

In [23]:
import torch.optim as optim
from ray import tune
from ray.tune.examples.mnist_pytorch import get_data_loaders, ConvNet, train, test


def train_mnist(config):
    train_loader, test_loader = get_data_loaders()
    model = ConvNet()
    optimizer = optim.SGD(model.parameters(), lr=config["lr"])
    for i in range(1000):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)
        ray.show_in_webui(str(acc))
        tune.track.log(mean_accuracy=acc)


analysis = tune.run(
    train_mnist, config={"lr": tune.grid_search([0.001, 0.01, 0.1])})

print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))

# Get a dataframe for analyzing trial results.
df = analysis.dataframe()

2020-01-20 16:28:44,211	INFO function_runner.py:250 -- tune.track signature detected.


Trial name,status,loc,lr
train_mnist_fb41e224,RUNNING,,
train_mnist_fb420150,PENDING,,
train_mnist_fb4220fe,PENDING,,


Result for train_mnist_fb4220fe:
  date: 2020-01-20_16-28-45
  done: false
  experiment_id: 24ca08e0bc604b6f9c36fa42244e3cf6
  experiment_tag: 2_lr=0.1
  hostname: Yunzhis-MacBook-Pro.local
  iterations_since_restore: 1
  mean_accuracy: 0.515625
  node_ip: 192.168.1.69
  pid: 92936
  time_since_restore: 0.2242109775543213
  time_this_iter_s: 0.2242109775543213
  time_total_s: 0.2242109775543213
  timestamp: 1579566525
  timesteps_since_restore: 0
  training_iteration: 0
  trial_id: fb4220fe
  
Result for train_mnist_fb41e224:
  date: 2020-01-20_16-28-45
  done: false
  experiment_id: cba156ca3cee46d48aad92fa3b3b0818
  experiment_tag: 0_lr=0.001
  hostname: Yunzhis-MacBook-Pro.local
  iterations_since_restore: 1
  mean_accuracy: 0.10625
  node_ip: 192.168.1.69
  pid: 92941
  time_since_restore: 0.26888275146484375
  time_this_iter_s: 0.26888275146484375
  time_total_s: 0.26888275146484375
  timestamp: 1579566525
  timesteps_since_restore: 0
  training_iteration: 0
  trial_id: fb41e224
 

Trial name,status,loc,lr,iter,total time (s),acc
train_mnist_fb41e224,RUNNING,192.168.1.69:92941,0.001,21,4.31757,0.375
train_mnist_fb420150,RUNNING,192.168.1.69:92938,0.01,21,4.3974,0.821875
train_mnist_fb4220fe,RUNNING,192.168.1.69:92936,0.1,21,4.30102,0.93125


Result for train_mnist_fb4220fe:
  date: 2020-01-20_16-28-50
  done: false
  experiment_id: 24ca08e0bc604b6f9c36fa42244e3cf6
  experiment_tag: 2_lr=0.1
  hostname: Yunzhis-MacBook-Pro.local
  iterations_since_restore: 27
  mean_accuracy: 0.88125
  node_ip: 192.168.1.69
  pid: 92936
  time_since_restore: 5.287028074264526
  time_this_iter_s: 0.1961829662322998
  time_total_s: 5.287028074264526
  timestamp: 1579566530
  timesteps_since_restore: 0
  training_iteration: 26
  trial_id: fb4220fe
  
Result for train_mnist_fb41e224:
  date: 2020-01-20_16-28-50
  done: false
  experiment_id: cba156ca3cee46d48aad92fa3b3b0818
  experiment_tag: 0_lr=0.001
  hostname: Yunzhis-MacBook-Pro.local
  iterations_since_restore: 27
  mean_accuracy: 0.415625
  node_ip: 192.168.1.69
  pid: 92941
  time_since_restore: 5.298834800720215
  time_this_iter_s: 0.19714879989624023
  time_total_s: 5.298834800720215
  timestamp: 1579566530
  timesteps_since_restore: 0
  training_iteration: 26
  trial_id: fb41e224
  


...

Trial name,status,loc,lr,iter,total time (s),acc
train_mnist_fb41e224,RUNNING,192.168.1.69:92941,0.001,919,175.919,0.91875
train_mnist_fb420150,RUNNING,192.168.1.69:92938,0.01,909,175.733,0.953125
train_mnist_fb4220fe,RUNNING,192.168.1.69:92936,0.1,913,175.784,0.984375


Result for train_mnist_fb41e224:
  date: 2020-01-20_16-31-43
  done: false
  experiment_id: cba156ca3cee46d48aad92fa3b3b0818
  experiment_tag: 0_lr=0.001
  hostname: Yunzhis-MacBook-Pro.local
  iterations_since_restore: 936
  mean_accuracy: 0.896875
  node_ip: 192.168.1.69
  pid: 92941
  time_since_restore: 178.95982789993286
  time_this_iter_s: 0.18797087669372559
  time_total_s: 178.95982789993286
  timestamp: 1579566703
  timesteps_since_restore: 0
  training_iteration: 935
  trial_id: fb41e224
  
Result for train_mnist_fb420150:
  date: 2020-01-20_16-31-44
  done: false
  experiment_id: cc2cc642d70941039b51910f0046719b
  experiment_tag: 1_lr=0.01
  hostname: Yunzhis-MacBook-Pro.local
  iterations_since_restore: 929
  mean_accuracy: 0.959375
  node_ip: 192.168.1.69
  pid: 92938
  time_since_restore: 179.3813738822937
  time_this_iter_s: 0.19555306434631348
  time_total_s: 179.3813738822937
  timestamp: 1579566704
  timesteps_since_restore: 0
  training_iteration: 928
  trial_id: fb4

Trial name,status,loc,lr,iter,total time (s),acc
train_mnist_fb41e224,RUNNING,192.168.1.69:92941,0.001,945,180.845,0.9125
train_mnist_fb420150,RUNNING,192.168.1.69:92938,0.01,936,180.904,0.940625
train_mnist_fb4220fe,RUNNING,192.168.1.69:92936,0.1,940,180.929,0.96875


Result for train_mnist_fb41e224:
  date: 2020-01-20_16-31-48
  done: false
  experiment_id: cba156ca3cee46d48aad92fa3b3b0818
  experiment_tag: 0_lr=0.001
  hostname: Yunzhis-MacBook-Pro.local
  iterations_since_restore: 963
  mean_accuracy: 0.915625
  node_ip: 192.168.1.69
  pid: 92941
  time_since_restore: 184.06556677818298
  time_this_iter_s: 0.18515467643737793
  time_total_s: 184.06556677818298
  timestamp: 1579566708
  timesteps_since_restore: 0
  training_iteration: 962
  trial_id: fb41e224
  
Result for train_mnist_fb420150:
  date: 2020-01-20_16-31-49
  done: false
  experiment_id: cc2cc642d70941039b51910f0046719b
  experiment_tag: 1_lr=0.01
  hostname: Yunzhis-MacBook-Pro.local
  iterations_since_restore: 956
  mean_accuracy: 0.9375
  node_ip: 192.168.1.69
  pid: 92938
  time_since_restore: 184.55940294265747
  time_this_iter_s: 0.19995594024658203
  time_total_s: 184.55940294265747
  timestamp: 1579566709
  timesteps_since_restore: 0
  training_iteration: 955
  trial_id: fb4

KeyboardInterrupt: 